In [2]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install transformers==4.31

In [4]:
!pip install -q einops
!pip install numpy==1.24
!pip install sentencepiece==0.1.98
#!pip install git+https://github.com/huggingface/transformers@v4.31
#!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [5]:
import os
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [6]:
model_name = "deepak2009thakur/Llama-2-7b-chat-finetune-rootcause"
model = AutoModelForCausalLM.from_pretrained(
model_name,
trust_remote_code=True,
torch_dtype=torch.bfloat16,
device_map= 'auto',
offload_folder='offload',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

In [7]:
!git clone https://github.com/ggerganov/llama.cpp.git

In [8]:
%cd llama.cpp

In [9]:
model.save_pretrained('./models/')

In [10]:
import os
import requests

def download_file_from_huggingface(model_name, filename, save_path):
  url = f"https://huggingface.co/{model_name}/resolve/main/{filename}"
  r = requests.get(url)
  if r.status_code != 200:

    print(f"Failed to download {filename}.HTTP Status Code: {r.status_code}")
    return False
  with open(os.path.join(save_path, filename), "wb") as f:
    f.write(r.content)
  return True

In [15]:
def main():
  files_to_download = [
      "tokenizer_config.json",
      "tokenizer.model",
      "tokenizer.json",
      "special_tokens_map.json",
      "added_tokens.json"
      ]
  # Create the directory if it doesn't exist
  save_path="./models"
  if not os.path.exists(save_path):
    os.makedirs(save_path)

  # Download the files
  for filename in files_to_download:
      success = download_file_from_huggingface(model_name, filename, save_path)
      if success:
        print(f"Successfully downloaded {filename}")
      else:
        print(f"Failed to download {filename}")

if __name__ == "__main__":
    main()

In [12]:
!apt update -y
!apt install build-essential git cmake libopenblas-dev libeigen3-dev -y

In [13]:
!make LLAMA_OPENBLAS=1

In [14]:
!python3 -m pip install -r requirements.txt

In [16]:
!python convert-hf-to-gguf.py models/

In [17]:
# Split the string by the '/' character
parts = model_name.split('/')

# Get the second part (index 1 because Python is 0-based)
model_name_pure = parts [1]

quant_type = "Q4_K"
quantized_model = f'models/{model_name_pure}.{quant_type}.gguf'

print(f'Preparing {quantized_model} with {quant_type} quantization.')


In [18]:
import subprocess
# Build the command as a list of arguments
command = ["./quantize", "models/ggml-model-f16.gguf", quantized_model, quant_type]
# Execute the command
subprocess.run(command)

CompletedProcess(args=['./quantize', 'models/ggml-model-f16.gguf', 'models/Llama-2-7b-chat-finetune-rootcause.Q4_K.gguf', 'Q4_K'], returncode=0)

In [20]:
from huggingface_hub import HfApi

api = HfApi()
quant_name = model_name.split('/') [-1]+"-GGUF" # Splits by '/' and takes the

In [21]:
quant_name

'Llama-2-7b-chat-finetune-rootcause-GGUF'

In [ ]:
"Ayaansh/TinyLlama-1.1B-Chat-v0.3-GGUF"

In [22]:
# Specify the repository where you want to upload the files
repo_id = "deepak2009thakur/" + quant_name
base_path = "./models"
# Array of local file paths you want to upload
local_file_paths = [
        base_path + "/special_tokens_map.json",
        base_path + "/tokenizer_config.json",
        base_path + "/tokenizer.json",
        base_path + "/tokenizer.model",
        base_path + "/ggml-vocab-llama-bpe.gguf",
        base_path + "/tokenizer.json",
        base_path + "/" + f'{model_name_pure}.{quant_type}.gguf',
        ]


In [23]:
# Loop through each file and upload it
for local_file_path in local_file_paths:
  # Extract the file name from the local file path
  file_name = local_file_path.split("/")[-1]
  print(file_name)
  # Specify the path where you want the file to be uploaded in the repository
  path_in_repo = file_name # Using file_name directly, adjust as needed
  #Upload the file
  api.upload_file(
                  path_or_fileobj=local_file_path,
                  path_in_repo=path_in_repo,
                  repo_id=repo_id,
                  repo_type="model",
  )
  print (f"Uploaded {file_name} to {repo_id}")

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

ggml-vocab-llama-bpe.gguf:   0%|          | 0.00/7.82M [00:00<?, ?B/s]

Llama-2-7b-chat-finetune-rootcause.Q4_K.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]